In [ ]:
!pip install -q mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [1]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os
import pandas as pd



def draw_landmarks_on_image(rgb_image, detection_result):

    #detect face landmarks for one or more face
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)
    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]

    #normalize face landmarks between 0 and 1
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks])

    #draw face landmarks and making connections between them
    #tesselation style is making the newtork of triangles on the face
    #contour style highlights the lips, eyes, eyebrows
    #irises style highlights the irises of the eyes

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

    return annotated_image , face_landmarks_proto



def getYPR(detection_result):
    R = detection_result.facial_transformation_matrixes[0][:3,:3]
    if abs(R[2, 0]) != 1:

          pitch = -np.arcsin(R[2, 0])  # theta (pitch)
          yaw = np.arctan2(R[1, 0], R[0, 0])  # psi (yaw)
          roll = np.arctan2(R[2, 1], R[2, 2])  # phi (roll)
    else:
        # Gimbal lock case
        if R[2, 0] == -1:
          pitch = np.pi / 2
          yaw = np.arctan2(-R[0, 1], R[0, 2])
          roll = 0
        else:
          pitch = -np.pi / 2
          yaw = np.arctan2(R[0, 1], R[0, 2])
          roll = 0

    pitch = pitch*180
    yaw = yaw*180
    roll = roll*180
    return yaw, pitch, roll

def drawYPRonIMG(face_landmarks_proto,yaw,pitch,roll):
    landmarks_data = [(lm.x, lm.y, lm.z) for lm in face_landmarks_proto.landmark]
    landmarks_array = np.array(landmarks_data)

    left_eyebrow_landmarks = [landmarks_array[i] for i in range(33, 133)]
    right_eyebrow_landmarks = [landmarks_array[i] for i in range(362, 464)]

    left_eyebrow_avg = np.mean(left_eyebrow_landmarks, axis=0)
    right_eyebrow_avg = np.mean(right_eyebrow_landmarks, axis=0)
    avg_position = ((left_eyebrow_avg[0] + right_eyebrow_avg[0]) / 2, (left_eyebrow_avg[1] + right_eyebrow_avg[1]) / 2)
    w, h, _ = img.shape
    avg_position = (int(avg_position[0] * w), int(avg_position[1] * h))


    center = avg_position
    length = 100
    line_thickness = 3


    yaw_end = (int(center[0] + length * np.cos(np.radians(yaw))),
               int(center[1] - length * np.sin(np.radians(yaw))))

    pitch_end = (int(center[0] + length * np.sin(np.radians(pitch))),
                 int(center[1] - length * np.cos(np.radians(pitch))))

    roll_end = (int(center[0] + length * np.cos(np.radians(roll))),
                int(center[1] - length * np.sin(np.radians(roll))))
    cv2.arrowedLine(img, center, yaw_end, (255,0,0), line_thickness)
    cv2.arrowedLine(img, center, pitch_end, (0,255,0), line_thickness)
    cv2.arrowedLine(img, center, roll_end, (0,0,255), line_thickness)

    cv2_imshow(img)




base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)
noface =0

posedata = pd.DataFrame(columns=['url', 'yaw','pitch','roll'])
DirPath = "/content/drive/MyDrive/AFLW2000"
Files = os.listdir(DirPath)
for File in Files:
    img_path = os.path.join(DirPath, File)
    img = mp.Image.create_from_file(img_path)
    detection_result = detector.detect(img)
    if detection_result.face_landmarks:
       yaw, pitch, roll = getYPR(detection_result)
       posedata.loc[len(posedata.index)] = [img_path, yaw, pitch, roll]


posedata.to_csv('/content/drive/MyDrive/data.csv')




ModuleNotFoundError: No module named 'mediapipe'